In [1]:
import os
import tensorflow as tf

from tqdm import tqdm
import pandas as pd
import numpy as np

import pathlib

In [2]:
# Download caption annotation files
annotation_folder = '/annotations/'
if not os.path.exists(os.path.abspath('.') + annotation_folder):
  annotation_zip = tf.keras.utils.get_file('captions.zip',
                                           cache_subdir=os.path.abspath('.'),
                                           origin='http://images.cocodataset.org/annotations/annotations_trainval2014.zip',
                                           extract=True)
  annotation_file = os.path.dirname(annotation_zip)+'/annotations/captions_train2014.json'
  os.remove(annotation_zip)

# Download image files
image_folder = '/train2014/'
if not os.path.exists(os.path.abspath('.') + image_folder):
  image_zip = tf.keras.utils.get_file('train2014.zip',
                                      cache_subdir=os.path.abspath('.'),
                                      origin='http://images.cocodataset.org/zips/train2014.zip',
                                      extract=True)
  PATH = os.path.dirname(image_zip) + image_folder
  os.remove(image_zip)
else:
  PATH = os.path.abspath('.') + image_folder

13510582272/13510573713 [==============================] - 1530s 0us/step


In [3]:
with open(annotation_file, 'r') as f:
    annotations = json.load(f)

NameError: name 'json' is not defined

In [8]:
path_to_file = pathlib.Path(path_to_zip).parent/'spa-eng/spa.txt'

In [33]:
Data = pd.read_csv(path_to_file, header = None, sep = "\t", encoding = "utf-8").dropna().reset_index(drop = True)

In [34]:
Data.columns = ['eng', "esp"]

In [35]:
source = list(Data.eng)
target = list(Data.esp)

In [64]:
source_02 = ['[CLS] ' + source[i].lower() + ' [SEP]' for i in range(len(source))]
target_02 = ['[CLS] ' + target[i].lower() + ' [SEP]' for i in range(len(target))]

In [77]:
processor = tf.keras.layers.TextVectorization(max_tokens=8002)

In [78]:
processor.adapt(source_02 + target_02)

In [113]:
vocab_size = len(processor.get_vocabulary()) +1
hidden_size = 64
batch_size = 64
max_length = 64
learning_rate = 0.001

In [89]:
source_03, target_03 = processor(source_02), processor(target_02)

In [91]:
source_04 =  tf.keras.preprocessing.sequence.pad_sequences(source_03, padding='post', maxlen = max_length)

In [92]:
target_04 = tf.keras.preprocessing.sequence.pad_sequences(target_03, padding='post', maxlen = max_length)

In [94]:
source_04.shape, target_04.shape

((118964, 64), (118964, 64))

In [98]:
Encoder_Input = tf.keras.Input(shape = (max_length))
Encoder_Embedding = tf.keras.layers.Embedding(vocab_size, hidden_size)(Encoder_Input)

Encoder_GRU = tf.keras.layers.GRU(hidden_size, activation='relu', return_sequences=False, dropout = 0.1)(Encoder_Embedding)

In [114]:
Decoder_Input = tf.keras.Input(shape = (max_length))
Decoder_Embedding = tf.keras.layers.Embedding(vocab_size, hidden_size)(Decoder_Input)

Decoder_output, Decoder_state = tf.keras.layers.GRU(hidden_size, activation='relu', return_sequences=True, return_state=True, dropout = 0.1)(Decoder_Embedding)

x = tf.keras.layers.Dense(vocab_size)(Decoder_output)

In [129]:
class Encoder(tf.keras.layers.Layer):
    def __init__(self, vocab_size, hidden_size, max_length):
        super(Encoder, self).__init__()
        self.vocab_size = vocab_size
        self.hidden_size = hidden_size
        self.max_length = max_length
        self.embedding = tf.keras.layers.Embedding(self.vocab_size, self.hidden_size)
        self.gru = tf.keras.layers.GRU(self.max_length, return_sequences=True, return_state=True, recurrent_initializer='glorot_uniform')
        
    def call(self, inputs):
        x = self.embedding(inputs)
        outputs, states = self.gru(x)
        
        return outputs, states

In [130]:
enc = Encoder(vocab_size, hidden_size, max_length)

In [131]:
outputs, states = enc(source_04[:4])
outputs

<tf.Tensor: shape=(4, 64, 64), dtype=float32, numpy=
array([[[ 0.00848125,  0.00416288, -0.00988456, ..., -0.00818877,
         -0.00200402, -0.01043381],
        [ 0.00575395,  0.00091073, -0.0180587 , ..., -0.01110739,
         -0.00548571,  0.00170174],
        [ 0.02157923, -0.00970342, -0.01733443, ..., -0.00873735,
         -0.01395188, -0.0032324 ],
        ...,
        [ 0.05305695,  0.01100166,  0.03185636, ..., -0.02992897,
          0.01193063,  0.01769838],
        [ 0.05305695,  0.01100166,  0.03185636, ..., -0.02992897,
          0.01193063,  0.01769838],
        [ 0.05305695,  0.01100166,  0.03185636, ..., -0.02992897,
          0.01193063,  0.01769838]],

       [[ 0.00848125,  0.00416288, -0.00988456, ..., -0.00818877,
         -0.00200402, -0.01043381],
        [ 0.00575395,  0.00091073, -0.0180587 , ..., -0.01110739,
         -0.00548571,  0.00170174],
        [ 0.02157923, -0.00970342, -0.01733443, ..., -0.00873735,
         -0.01395188, -0.0032324 ],
        ...,
 

In [132]:
states

<tf.Tensor: shape=(4, 64), dtype=float32, numpy=
array([[ 5.3056948e-02,  1.1001656e-02,  3.1856358e-02,  1.7256709e-03,
        -1.1928906e-02, -3.7495859e-04, -1.3896550e-02,  1.9561470e-02,
         1.8687159e-02,  1.1892814e-02, -2.6391979e-02,  5.1784851e-02,
         2.0139676e-02,  2.8082062e-02, -4.3983545e-02, -1.1873415e-02,
         3.9360162e-02,  3.8132682e-02,  2.9752320e-02,  2.3796085e-02,
        -2.8523874e-02, -9.4494233e-03, -2.0592460e-02,  1.0059174e-02,
        -2.0227259e-02,  4.2502880e-03, -5.2545192e-03, -6.3916352e-03,
        -9.0982992e-04,  1.1030405e-02, -1.3762703e-03, -9.5622297e-03,
         2.8065327e-03,  7.1357489e-03, -5.2410131e-03,  3.7013933e-02,
         2.9182542e-02,  1.6783884e-02,  2.3554252e-02, -2.4742110e-02,
        -2.7179727e-02,  2.6684754e-02,  5.2712155e-03, -3.0241318e-02,
         2.5377160e-02,  1.7509429e-02,  2.8006691e-03,  2.4983410e-02,
        -1.7965337e-02,  3.0183582e-02,  1.0294854e-02,  1.1469036e-02,
        -3.1514

In [139]:
class Decoder(tf.keras.layers.Layer):
    def __init__(self, vocab_size, hidden_size, max_length):
        super(Decoder, self).__init__()
        self.vocab_size = vocab_size
        self.hidden_size = hidden_size
        self.max_length = max_length
        self.embedding = tf.keras.layers.Embedding(self.vocab_size, self.hidden_size)
        self.gru = tf.keras.layers.GRU(self.max_length, return_sequences=True, return_state=True, recurrent_initializer='glorot_uniform')
        self.attention = tf.keras.layers.Attention()
        self.wc = tf.keras.layers.Dense(self.max_length)
        self.fc = tf.keras.layers.Dense(self.vocab_size)
        
    def call(self, inputs, enc_ouputs = None, return_state=False, training=False):
        x = self.embedding(inputs)

        if enc_ouputs is None:
            enc_ouputs = self.gru.get_initial_state(x)        
        outputs, states = self.gru(x, initial_state=enc_ouputs, training=training)
        x = self.wc(outputs)
        x = self.fc(x)
        return x

In [140]:
dec = Decoder(vocab_size, hidden_size, max_length)

In [141]:
dec(target_04[:4], states)

<tf.Tensor: shape=(4, 64, 8003), dtype=float32, numpy=
array([[[ 2.1982929e-03, -1.0801135e-03, -1.0143860e-03, ...,
         -2.4488190e-04,  1.9925400e-03,  1.3355419e-05],
        [ 1.4968932e-03, -4.7003297e-04, -2.0716371e-04, ...,
         -1.0709614e-03, -6.4194429e-04, -7.5892010e-04],
        [-6.0943334e-04, -4.8160538e-04, -8.2564744e-05, ...,
          1.0857458e-03, -3.2275948e-03, -3.5852867e-05],
        ...,
        [ 2.6121561e-03,  1.8557305e-03,  3.2420403e-03, ...,
          3.2647589e-04, -2.6801793e-04,  2.8835847e-03],
        [ 2.6121561e-03,  1.8557305e-03,  3.2420403e-03, ...,
          3.2647589e-04, -2.6801793e-04,  2.8835847e-03],
        [ 2.6121561e-03,  1.8557305e-03,  3.2420403e-03, ...,
          3.2647589e-04, -2.6801793e-04,  2.8835847e-03]],

       [[ 2.1982929e-03, -1.0801135e-03, -1.0143860e-03, ...,
         -2.4488190e-04,  1.9925400e-03,  1.3355419e-05],
        [ 2.0087957e-03, -2.0758607e-03, -1.4774778e-03, ...,
          2.6493722e-03,  1.

In [101]:
Decoder_output

<KerasTensor: shape=(None, 64, 64) dtype=float32 (created by layer 'gru_6')>

In [ ]:

Layer01_RNN_Forward = tf.keras.layers.SimpleRNN(hidden_size, return_sequences=False, go_backwards=False)(Embedding_Layers)
Classifier = tf.keras.layers.Dense(1, activation = 'sigmoid')(Layer01_RNN_Forward)